<a href="https://colab.research.google.com/github/colinrsmall/ehm_roster_tools/blob/master/Fantrax_Face_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import requests
import json
import urllib.request
from itertools import permutations
from datetime import datetime
from tqdm import tqdm

url = 'https://www.fantrax.com/fxpa/req'
alphabet = 'abcdefghijklmnopqrstuvwxyz0123456789'
prev_hit = False
prev = '03yz9'

for prefix in ['0', '1', '2', '3', '4', '5']:
    for code in tqdm(permutations(alphabet, 3), total=42840, unit="players"):
        if(f'0{prefix}{"".join(code)}' == prev):
            prev_hit = True
        if(prev_hit):
            payload = '{"msgs":[{"method":"getPlayerProfile","data":{"playerId":"' + f'0{prefix}{"".join(code)}' + '"}},{"method":"getRefObject","data":{"type":"PlayerActionType"}}],"ng2":true,"href":"https://www.fantrax.com/player/00400","dt":1,"at":0,"tz":"America/New_York","v":"8.0.3"}'
            headers = {}
            response = requests.request('POST', url, headers = headers, data = payload)
            if response.status_code == 200:    
                response_json = json.loads(response.text)
                try:
                    player_position = response_json['responses'][0]['data']['miscData']['defaultPosition']
                    headshot_url = response_json['responses'][0]['data']['miscData']['headshotUrl']
                    if( headshot_url != "null" ):
                        first_name = response_json['responses'][0]['data']['miscData']['name'].split(',')[1].strip()
                        last_name = response_json['responses'][0]['data']['miscData']['name'].split(',')[0].strip()
                        birthdate = datetime.strptime(response_json['responses'][0]['data']['miscData']['personalInfo'][1]['value'], '%B %d, %Y').strftime('%d.%m.%Y')
                        image = urllib.request.urlopen(headshot_url).read()
                        if(len(image) > 0):
                            if (player_position in ['C', 'RW', 'LW', "G", "D"]):
                                with open(f'faces/{first_name}_{last_name}_{birthdate}.png', 'wb+') as image_file:
                                    image_file.write(image)
                                    image_file.close()
                            print(f"Code: 0{prefix}{''.join(code)} - {first_name} {last_name}")
                except Exception as e:
                    print(f'\n0{prefix}{"".join(code)}')
                    print(f'{e}')
            else:
                print(f"Response {response.status_code} for code {prefix}{''.join(code)}")


sys.exit(1)